In [ ]:
from utils import mysettings
als_lab_postcodes_dict = mysettings.ALS_LABS_POSTCODES_DIC
als_lab_postcodes_dict

In [2]:
combine_folder_path = mysettings.COMBINED_FOLDER_PATH
combine_folder_path

'data/pre_processed_combined/sales'

In [3]:
from tqdm import tqdm
from pathlib import Path
import pandas as pd

In [4]:
def split_data(df, column_name):
    # Create two DataFrames: one with null values, one without
    df_untagged_nhs = df[df[column_name].isnull()]  # Rows where column_name is NULL
    df_tagged_nhs = df[df[column_name].notnull()]  # Rows where column_name is NOT NULL

    return df_untagged_nhs, df_tagged_nhs

In [5]:
def nhs_tag_as_als_lab (df,search_list,column_name):    
    if "practice_name"  in df.columns:
        df["practice_name"] = df["practice_name"].str.lower()
        
    if "customer_name"in df.columns:
        df["customer_name"] = df["customer_name"].str.lower()

    for substring in search_list:  
        if "practice_name"  in df.columns:            
            df.loc[(df["practice_name"].str.contains(substring, na=False)), column_name] = "ALS Lab"
        if "customer_name"  in df.columns:            
            df.loc[(df["customer_name"].str.contains(substring, na=False)), column_name] = "ALS Lab"


    return split_data(df,column_name)

In [ ]:
def nhs_tag_ashford(df, column_name):
        # Add in NHS and private tags from the Ashford prices list
        ashford_price_list = mysettings.ASHFORD_PRICE_LIST
        ashford_price_list_df = pd.read_excel(ashford_price_list)

        ashford_price_list_df["ProductID"] = ashford_price_list_df["ProductID"].astype(str)
        df = df.merge(
                ashford_price_list_df, left_on="product_code", right_on="ProductID", how="left"
        )
        df[column_name] = df[column_name].fillna(
                df["Class"]
        )

        df.loc[
        df[column_name].isna() == True, column_name] = "Unknown"

        # Drop extraneous columns
        df = df.drop(
                columns=["ProductID", "Description", "Price 2024", "Class"]
        )

        return split_data(df,column_name)

In [9]:
combine_folder_path = Path(combine_folder_path)
tagged_data_list = []
for file in tqdm(combine_folder_path.rglob("*")):
        print(f"processing NHS tagging for : {str(file)}") 
        
        combined_data_df = pd.read_csv(file)
        column_name = "nhs_or_private"

        if column_name not in combined_data_df.columns:
                combined_data_df[column_name] = None

        untagged_data, tagged_data =split_data(combined_data_df, column_name)  
        print(f"file level 1 [{file}]Total number {len(combined_data_df)} /n untagged [{len(untagged_data)}] /n tagged [{len(tagged_data)}]")
        # if len(tagged_data) > 0:
        #         grouped_df = tagged_data.groupby(column_name).size().reset_index(name="count")
        #         print(grouped_df)

        total_rows_for_untagged_data = len(untagged_data)
        als_names_search_list = mysettings.ALS_NAME_SEARCH_TERM
        untagged_data, tagged_data = (nhs_tag_as_als_lab(untagged_data, als_names_search_list, column_name))
        print(f"file level 2 [{file}]Total number {total_rows_for_untagged_data} /n untagged [{len(untagged_data)}] /n tagged [{len(tagged_data)}]")

        #check if untagged data is ashford
        total_rows_for_untagged_data = len(untagged_data)
        untagged_data_ashford = untagged_data.loc[untagged_data["als_lab"].str.lower() == "ashford"]
        untagged_data = untagged_data.loc[untagged_data["als_lab"].str.lower() != "ashford"]

        print (f"length of untagged ashford is {len(untagged_data_ashford)} while untagged is {len(untagged_data)}")
        if len(untagged_data_ashford) > 0:
                untagged_data_ashford, tagged_data = nhs_tag_ashford(untagged_data_ashford, column_name)
                untagged_data=pd.concat([untagged_data,untagged_data_ashford])
        print(f"file level 3 [{file}]Total number {total_rows_for_untagged_data} /n untagged [{len(untagged_data)}] /n tagged [{len(tagged_data)}]")
        
        


        


0it [00:00, ?it/s]

processing NHS tagging for : data\pre_processed_combined\sales\combined_Ashford.csv


C:\Users\Admin\AppData\Local\Temp\ipykernel_2340\3739307741.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_data_df = pd.read_csv(file)


file level 1 [data\pre_processed_combined\sales\combined_Ashford.csv]Total number 978624 /n untagged [978624] /n tagged [0]
file level 2 [data\pre_processed_combined\sales\combined_Ashford.csv]Total number 978624 /n untagged [972657] /n tagged [5967]
length of untagged ashford is 972657 while untagged is 0


1it [00:17, 17.13s/it]

file level 3 [data\pre_processed_combined\sales\combined_Ashford.csv]Total number 972657 /n untagged [171321] /n tagged [801336]


In [ ]:
combined_data_df = pd.read_excel("data\pre_processed_combined\sales\Book1.xlsx")
column_name = "nhs_or_private"

combined_data_df.head(4)